# Build

This notebook builds and publishes a new version of Cirrus.

In [ ]:
# This cell causes all function definitions, method definitions, etc. to be reloaded each time a cell is run. It is
#    here for ease of development.
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import sys
import atexit

from cirrus import automate

In [ ]:
# Cirrus produces logs, but they will not show unless we add a handler that prints.
log = logging.getLogger("cirrus")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

# Cleanup
If you run into any problems, the following cell should clean up all AWS resources that were created.

In [ ]:
atexit._run_exitfuncs()

## Ubuntu
Cirrus needs to be built under Ubuntu in order to create an executable for the parameter server.

### Build image
The first step is to create a "build image", which is a machine image that has the proper environment for building Cirrus set up on it. This step is separated from the following step because it rarely needs to be re-run.

In [ ]:
automate.make_ubuntu_build_image("cirrus_ubuntu_build_image")

### Executables
Now, we compile Cirrus on the build image and publish its executables.

Only Joao's AWS account has access to the `cirrus-public` bucket. If this cell is run under another account, it will fail.

In [ ]:
automate.make_executables("s3://cirrus-public/executables", "cirrus_ubuntu_build_image", "ubuntu")